# Experiment with yolo models to obtain better football detection

## Imports

In [ ]:
import os

import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.path.dirname(os.getcwd()),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

if os.getenv("DATA_ROOT") is None:
    os.environ["DATA_ROOT"] = f"{root}"

In [ ]:
from pathlib import Path

import cv2
import hydra
import supervision as sv

In [ ]:
import torch
from hydra import compose, initialize

# Setup device-agnostic code
if torch.cuda.is_available():
    DEVICE = "cuda"  # NVIDIA GPU
    print("GPU Found!!")
else:
    raise Exception("No GPU Found!!")

## Paths setup

In [ ]:
from omegaconf import DictConfig

with initialize(config_path="../configs", job_name="EDA", version_base=None):
    cfg: DictConfig = compose(config_name="train.yaml")
    # print(OmegaConf.to_yaml(cfg))
    print(cfg.paths)

In [ ]:
cfg.paths.pretrained_model_dir

In [ ]:
# auto reload libs
%load_ext autoreload
%autoreload 2

## Download Datasets

In [ ]:
ROOT_DIR = Path(cfg.paths.root_dir)
ROOT_DATA_DIR = Path(cfg.paths.root_data_dir)
DATA_DIR = ROOT_DATA_DIR / cfg.datasets.datasets_dir
DATASET = cfg.datasets.roboflow
DATA_DIR.mkdir(parents=True, exist_ok=True)
DATASET_DIR = DATA_DIR / cfg.datasets.dataset_name
CLASS_NAMES = cfg.datasets.names
PRETRAINED_MODEEL_DIR = Path(cfg.paths.pretrained_model_dir)

In [ ]:
if len(list(DATA_DIR.iterdir())) == 0:
    from roboflow import Roboflow

    rf = Roboflow()
    project = rf.workspace(DATASET.workspace).project(DATASET.project)
    version = project.version(DATASET.version)
    dataset = version.download(model_format=DATASET.model)

## Paths Setup

## YOLO model import

In [ ]:
MODEL_NAME = f"{PRETRAINED_MODEEL_DIR}/{cfg.models.model_name.lower()}.pt"
MODEL_NAME

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL_NAME)

In [ ]:
cfg.paths.results_dir, cfg.task_name

In [ ]:
results = model.train(
    data=cfg.datasets.dataset_yaml,
    **cfg.trainer.params,
    **cfg.datasets.augmentations,
)

In [ ]:
results

In [ ]:
# YOLO11l summary (fused): 190 layers, 25,282,396 parameters, 0 gradients, 86.6 GFLOPs
#                  Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.12it/s]
#                    all         49       1174      0.814       0.73      0.791      0.516
#                   ball         45         45      0.701       0.26      0.341       0.14
#             goalkeeper         38         39      0.791      0.846      0.917      0.615
#                 player         49        973      0.932       0.95      0.981      0.731
#                referee         49        117      0.831      0.863      0.926      0.579

In [ ]:
# YOLOv12l summary (fused): 283 layers, 26,342,156 parameters, 0 gradients, 88.6 GFLOPs
#                  Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.92it/s]
#                    all         49       1174       0.86       0.75       0.81      0.537
#                   ball         45         45      0.783      0.333      0.415      0.154
#             goalkeeper         38         39      0.837      0.872      0.908      0.622
#                 player         49        973      0.964      0.928      0.985      0.755
#                referee         49        117      0.856      0.865      0.932      0.616

In [ ]:
# model = YOLO('/workspaces/football-players-tracking-yolo/results/augumented-data-yolo12l/weights/best.pt')
# results = model.train(data=cfg.datasets.dataset_yaml,
#             **cfg.trainer.params,
#             **cfg.datasets.augmentations,
# )